# Outlook Mail Reader Example

This notebook demonstrates how to use the `OutlookMailReader` class to search and filter emails from Microsoft Outlook using the Graph API.

## 1. Import and Initialize

In [ ]:
# Import the OutlookMailReader class
from outlook_mail_reader import OutlookMailReader

In [ ]:
# Option 1: Initialize with credentials from environment variables (.env file)
reader = OutlookMailReader(load_env=True)

In [ ]:
# Option 2: Initialize with explicit credentials
# reader = OutlookMailReader(
#     tenant_id="your-tenant-id",
#     client_id="your-client-id",
#     verbose=True
# )

In [ ]:
# Option 3: Initialize with an existing access token
# reader = OutlookMailReader(
#     access_token="your-access-token",
#     verbose=True
# )

## 2. Authenticate

If you provided an access token, this step is optional. Otherwise, you need to authenticate using the device code flow.

In [ ]:
# Authenticate (will display a URL and code to enter in your browser)
reader.authenticate()

## 3. Search for Emails

In [ ]:
# Search for emails from a specific sender containing a keyword
emails = reader.search_emails(
    sender_email="FASTRAPP@paypal.com",  # Sender email address
    keyword="failed",                    # Keyword to search for
    max_items=25,                        # Maximum number of emails to retrieve
    search_in_subject=True,              # Search in subject
    search_in_body=True,                 # Search in body
    case_sensitive=False,                # Case-insensitive search
    format_results=True                  # Format results into a cleaner structure
)

print(f"Found {len(emails)} matching emails")

## 4. Display Results

In [ ]:
# Display email information in a table
import pandas as pd

if emails:
    # Create a DataFrame for display
    df = pd.DataFrame([
        {
            'Date': email['timestamp'],
            'Sender': email['sender_email'],
            'Subject': email['subject'],
            'Link': f"[Open in Outlook]({email['outlook_link']})"
        }
        for email in emails
    ])
    
    # Display as HTML table
    display(df)
else:
    print("No matching emails found")

## 5. Examine Individual Emails

In [ ]:
# Print detailed information about the first email
if emails:
    reader.print_message_details(emails[0])

## 6. Advanced Usage: Custom Filtering

In [ ]:
# Get raw messages from sender
raw_messages = reader.get_messages_from_sender(
    sender_email="FASTRAPP@paypal.com",
    max_items=50
)

# Apply custom filtering logic
filtered_messages = []
for msg in raw_messages:
    subject = msg.get('subject', '').lower()
    body = msg.get('bodyPreview', '').lower()
    
    # Custom filter: emails containing 'failed' and 'authentication' together
    if ('failed' in subject or 'failed' in body) and ('authentication' in subject or 'authentication' in body):
        filtered_messages.append(msg)

# Format the filtered messages
formatted_messages = [reader.format_message_info(msg) for msg in filtered_messages]

print(f"Found {len(formatted_messages)} emails matching custom criteria")

## 7. Save Results to File

In [ ]:
# Save results to CSV
if emails:
    df = pd.DataFrame([
        {
            'Date': email['timestamp'],
            'Sender': email['sender_email'],
            'Subject': email['subject'],
            'Preview': email['preview'],
            'Link': email['outlook_link']
        }
        for email in emails
    ])
    
    # Save to CSV
    df.to_csv('fastrapp_failed_emails.csv', index=False)
    print("Results saved to 'fastrapp_failed_emails.csv'")